In [29]:
#!apt install -y scons libeigen3-dev
# need to replace SConstruct with modified version
"""
  sudo apt-get install libeigen3-dev scons \
    cmake libgtest-dev  # For testing.
  export EIGEN_PATH=/usr/include/eigen3
  export GTEST_SOURCE=/usr/src/gtest
"""
"""
apt install mercurial
hg clone https://bitbucket.org/MartinFelis/eigen3swig
apt install python3-numpy-dev
"""

'\n  sudo apt-get install libeigen3-dev scons     cmake libgtest-dev  # For testing.\n  export EIGEN_PATH=/usr/include/eigen3\n  export GTEST_SOURCE=/usr/src/gtest\n'

In [14]:
# apt install swig

"""
%module module
%{
#include header.h
%}
%include header.h
"""

def build_swig():
    swig -python module.i
    swig -python


SyntaxError: invalid syntax (<ipython-input-14-29e5453a193b>, line 4)

In [14]:
# based on: https://github.com/rdeits/swig-eigen-numpy

import os

data = """
%module carfac
%{
#include "carfac/cpp/carfac.h"
%}

%include <typemaps.i>
%include "eigen3swig/eigen.i"

// Templates for Eigen - Eigen is templating and swig does not create types for templates by itself
%eigen_typemaps(CARFAC::FPType)
%eigen_typemaps(CARFAC::ArrayXX)
%eigen_typemaps(CARFAC::ArrayXXf)

/////////// MANUAL EXPANSION OF EIGENTYPEMAPS for Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>
// Argout: const & (Disabled and prevents calling of the non-const typemap)
%typemap(argout, fragment="Eigen_Fragments") const Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> & ""

// Argout: & (for returning values to in-out arguments)
%typemap(argout, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> &
{
  // Argout: &
  CopyFromEigenToNumPyMatrix<Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>>($input, $1);
}

// In: (nothing: no constness)
%typemap(in, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> (Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> temp)
{
  ConvertFromNumpyToEigenMatrix<Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>>(&temp, $input);
  $1 = temp;
}
// In: const&
%typemap(in, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> const& (Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> temp)
{
  // In: const&
  ConvertFromNumpyToEigenMatrix<Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>>(&temp, $input);
  $1 = &temp;
}
// In: & (not yet implemented)
%typemap(in, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> & (Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> temp)
{
  // In: non-const&
  ConvertFromNumpyToEigenMatrix<Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>>(&temp, $input);

  $1 = &temp;
}
// In: const* (not yet implemented)
%typemap(in, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> const*
{
  PyErr_SetString(PyExc_ValueError, "The input typemap for const pointer is not yet implemented. Please report this problem to the developer.");
}
// In: * (not yet implemented)
%typemap(in, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> *
{
  PyErr_SetString(PyExc_ValueError, "The input typemap for non-const pointer is not yet implemented. Please report this problem to the developer.");
}

// Out: (nothing: no constness)
%typemap(out, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>
{
  ConvertFromEigenToNumPyMatrix<Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>>(&$result, &$1);
}
// Out: const
%typemap(out, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> const
{
  ConvertFromEigenToNumPyMatrix<Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>>(&$result, &$1);
}
// Out: const&
%typemap(out, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> const&
{
  ConvertFromEigenToNumPyMatrix<Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic>>(&$result, $1);
}
// Out: & (not yet implemented)
%typemap(out, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> &
{
  PyErr_SetString(PyExc_ValueError, "The output typemap for non-const reference is not yet implemented. Please report this problem to the developer.");
}
// Out: const* (not yet implemented)
%typemap(out, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> const*
{
  PyErr_SetString(PyExc_ValueError, "The output typemap for const pointer is not yet implemented. Please report this problem to the developer.");
}
// Out: * (not yet implemented)
%typemap(out, fragment="Eigen_Fragments") Eigen::Matrix<CARFAC::FPType, Eigen::Dynamic, Eigen::Dynamic> *
{
  PyErr_SetString(PyExc_ValueError, "The output typemap for non-const pointer is not yet implemented. Please report this problem to the developer.");
}

// include CARFAC library
%include "carfac/cpp/carfac.h"
%include "carfac/cpp/agc.h"
%include "carfac/cpp/car.h"
%include "carfac/cpp/common.h"
%include "carfac/cpp/ihc.h"

// TODO: should I template functions on CARFAC?
//%template(CARFAC_RunSegmentF) CARFAC::RunSegment<CARFAC::FPType>;

//typedef CARFAC::FPType FPType;
//typedef Eigen::Array<FPType, Eigen::Dynamic, 1> ArrayX;
//typedef Eigen::Array<FPType, Eigen::Dynamic, Eigen::Dynamic> ArrayXX;

%init %{
import_array();
%}

/// Sample method that accepts numpy arrays and gets eigen arrays
%{
PyObject* _wrap_CARFAC_makeArray(PyObject *SWIGUNUSEDPARM(self), PyObject *args){
  PyObject *resultobj = 0;
  PyObject * obj0 = 0 ;
  PyObject * obj1 = 0 ;
  int res1 = 0;
  void *local_void_ptr = 0;
  CARFAC *local_carfac = (CARFAC *) 0;
  
    //parse arguments
    if(!PyArg_ParseTuple(args,(char *)"OO:CARFAC_makeArray", &obj0, &obj1))
      SWIG_fail; /// Incorrect number of args and generally wrong arg tuple

    //cast type of first argument
    res1 = SWIG_ConvertPtr(obj0, &local_void_ptr, SWIGTYPE_p_CARFAC, 0 | 0);
    if (!SWIG_IsOK(res1)) {
        SWIG_exception_fail(SWIG_ArgError(res1), "in method '" "CARFAC_RunSegment" "', argument " "1"" of type '" "CARFAC *""'"); 
    }
    local_carfac = reinterpret_cast<CARFAC *>(local_void_ptr);
  
  /// Actual cast...
  //(arg1)->RunSegment((ArrayXX const &)*arg2,arg3,arg4);

      resultobj = SWIG_Py_Void();
      return resultobj;
    fail: // essentially an exception
      return NULL;
}
%}



/*%{
/// TAKEN from swig generation
SWIGINTERN PyObject *_wrap_CARFAC_RunSegment(PyObject *SWIGUNUSEDPARM(self), PyObject *args) {
  PyObject *resultobj = 0;
  CARFAC *arg1 = (CARFAC *) 0 ;
  ArrayXX *arg2 = 0 ;
  bool arg3 ;
  CARFACOutput *arg4 = (CARFACOutput *) 0 ;
  void *argp1 = 0 ;
  int res1 = 0 ;
  void *argp2 = 0 ;
  int res2 = 0 ;
  bool val3 ;
  int ecode3 = 0 ;
  void *argp4 = 0 ;
  int res4 = 0 ;
  PyObject * obj0 = 0 ;
  PyObject * obj1 = 0 ;
  PyObject * obj2 = 0 ;
  PyObject * obj3 = 0 ;
  
  if (!PyArg_ParseTuple(args,(char *)"OOOO:CARFAC_RunSegment",&obj0,&obj1,&obj2,&obj3)) SWIG_fail;
  res1 = SWIG_ConvertPtr(obj0, &argp1,SWIGTYPE_p_CARFAC, 0 |  0 );
  if (!SWIG_IsOK(res1)) {
    SWIG_exception_fail(SWIG_ArgError(res1), "in method '" "CARFAC_RunSegment" "', argument " "1"" of type '" "CARFAC *""'"); 
  }
  arg1 = reinterpret_cast< CARFAC * >(argp1);
  res2 = SWIG_ConvertPtr(obj1, &argp2, SWIGTYPE_p_Eigen__ArrayT_FPType_Eigen__Dynamic_Eigen__Dynamic_t,  0  | 0);
  if (!SWIG_IsOK(res2)) {
    SWIG_exception_fail(SWIG_ArgError(res2), "in method '" "CARFAC_RunSegment" "', argument " "2"" of type '" "ArrayXX const &""'"); 
  }
  if (!argp2) {
    SWIG_exception_fail(SWIG_ValueError, "invalid null reference " "in method '" "CARFAC_RunSegment" "', argument " "2"" of type '" "ArrayXX const &""'"); 
  }
  arg2 = reinterpret_cast< ArrayXX * >(argp2);
  ecode3 = SWIG_AsVal_bool(obj2, &val3);
  if (!SWIG_IsOK(ecode3)) {
    SWIG_exception_fail(SWIG_ArgError(ecode3), "in method '" "CARFAC_RunSegment" "', argument " "3"" of type '" "bool""'");
  } 
  arg3 = static_cast< bool >(val3);
  res4 = SWIG_ConvertPtr(obj3, &argp4,SWIGTYPE_p_CARFACOutput, 0 |  0 );
  if (!SWIG_IsOK(res4)) {
    SWIG_exception_fail(SWIG_ArgError(res4), "in method '" "CARFAC_RunSegment" "', argument " "4"" of type '" "CARFACOutput *""'"); 
  }
  arg4 = reinterpret_cast< CARFACOutput * >(argp4);
  (arg1)->RunSegment((ArrayXX const &)*arg2,arg3,arg4);
  resultobj = SWIG_Py_Void();
  return resultobj;
fail:
  return NULL;
}

%}*/
"""

with open("carfac.i", "w") as f:
    f.write(data)
    
!swig -python -c++ carfac.i
!g++ -fPIC -shared -std=c++11 carfac/cpp/sai.cc carfac/cpp/ear.cc carfac/cpp/carfac.cc -I/usr/include/eigen3  -Doverride= -o carfac/cpp/all.so
!g++ -c -fPIC -std=c++11 carfac_wrap.cxx -I/usr/include/python3.5 -I/usr/include/eigen3 -o carfac_wrap.o
!g++ -shared -fPIC carfac_wrap.o carfac/cpp/all.so -o _carfac.so
#!g++ -shared carfac_wrap.o carfac/cpp/carfac.o carfac/cpp/binaural_sai.o carfac/cpp/ear.o carfac/cpp/sai.o -o _carfac.so


In file included from /usr/include/python3.5/numpy/ndarraytypes.h:1777:0,
                 from /usr/include/python3.5/numpy/ndarrayobject.h:18,
                 from /usr/include/python3.5/numpy/arrayobject.h:4,
                 from carfac_wrap.cxx:3145:
/usr/include/python3.5/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^


In [1]:
import carfac
import numpy as np

In [2]:
params = carfac.CARParams()
# [getattr(params, x) for x in dir(params) if not x.startswith("_")]
ihc = carfac.IHCParams()
agc = carfac.AGCParams()
filters = carfac.CARFAC(2, 16000, params, ihc, agc)
output = carfac.CARFACOutput(True, True, True, True)
a = np.random.random([6]).astype(np.float32)
filters.RunSegment(a, False, output)

TypeError: in method 'CARFAC_RunSegment', argument 2 of type 'ArrayXX const &'

In [3]:
carfac.CARFAC??

Object `carfac.ArrayXX` not found.


In [6]:
cat carfac_wrap.cxx

/* ----------------------------------------------------------------------------
 * This file was automatically generated by SWIG (http://www.swig.org).
 * Version 3.0.8
 *
 * This file is not intended to be easily readable and contains a number of
 * coding conventions designed to improve portability and efficiency. Do not make
 * changes to this file unless you know what you are doing--modify the SWIG
 * interface file instead.
 * ----------------------------------------------------------------------------- */


#ifndef SWIGPYTHON
#define SWIGPYTHON
#endif

#define SWIG_PYTHON_DIRECTOR_NO_VTABLE


#ifdef __cplusplus
/* SwigValueWrapper is described in swig.swg */
template<typename T> class SwigValueWrapper {
  struct SwigMovePointer {
    T *ptr;
    SwigMovePointer(T *p) : ptr(p) { }
    ~SwigMovePointer() { delete ptr; }
    SwigMovePointer& operator=(SwigMovePointer& rhs) { T* oldptr = ptr; ptr = 0; delete oldptr; ptr = rhs.ptr; rhs.ptr = 0; return *this; }
  } pointer;
  SwigValue

In [9]:
cat carfac.py

# This file was automatically generated by SWIG (http://www.swig.org).
# Version 3.0.8
#
# Do not make changes to this file unless you know what you are doing--modify
# the SWIG interface file instead.





from sys import version_info
if version_info >= (2, 6, 0):
    def swig_import_helper():
        from os.path import dirname
        import imp
        fp = None
        try:
            fp, pathname, description = imp.find_module('_carfac', [dirname(__file__)])
        except ImportError:
            import _carfac
            return _carfac
        if fp is not None:
            try:
                _mod = imp.load_module('_carfac', fp, pathname, description)
            finally:
                fp.close()
            return _mod
    _carfac = swig_import_helper()
    del swig_import_helper
else:
    import _carfac
del version_info
try:
    _swig_property = property
except NameError:
    pass  # Python < 2.2 doesn't have 'property'.


def _swig_setattr_nondynamic(self, class_type,